In [45]:
import urllib.request
from bs4 import BeautifulSoup
import warnings
import datetime
warnings.simplefilter("ignore")
from datetime import datetime

from collections import namedtuple
import pickle
import pandas as pd
import numpy as np
from itertools import product

from selenium import webdriver
from time import sleep
import json
from math import radians, cos, sin, asin, sqrt
import copy

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing

import os
import folium

from IPython.core.display import display, HTML
from tqdm import tqdm
from branca.colormap import linear
import json
from folium import FeatureGroup, LayerControl, Map, Marker

In [2]:
Station = namedtuple('Station', 'name distance')
Organization = namedtuple('Organization', 'name rating reviews mean_check categories address')

In [3]:
class ImportantObject():
    def __init__(self, name, rating, reviews, mean_check, categories, link,  address):
        self.name = name
        self.rating = rating # "Народная" оценка уровня ослуживания 
        self.reviews = reviews
        self.lat, self.long = None, None # Широта и долгота
        self.categories = categories  # Типы объекта по внутренней классификации Яндекс.Карт
        self.link = link # Ссылка на объект 
        self.address = address  # Адрес  
        
    def __str__(self):
        return ("name = {name}, /n"
                "rating = {rating}, /n"
                "reviews = {reviews},/n"
                "lat = {lat}, /n"
                "long = {long}, /n" 
                "categories = {categories} /n"  
                "link = {link} /n"
                "address = {address} /n").format(name= self.name,
                                                rating = self.rating,
                                                reviews = self.reviews,
                                                lat = self.lat,
                                                long = self.long,
                                                categories = self.categories,
                                                link = self.link,
                                                address = self.address)

In [4]:
class WrapperIO:
    def __init__(self, io):
        self.io = io 
        
    def __eq__(self, wio):
        return (self.io.name == wio.io.name) and (self.io.address == wio.io.address)
    
    def __hash__(self):
        return hash(self.io.name)^hash(self.io.address)

# Визуализация данных

In [5]:
LAT = 55.389
LONG = 37.056 

In [6]:
def get_id(lat, long):
    row = (lat - LAT)//0.009 + 1
    column = (long-LONG)//0.016 
    id_ = row*64+column
    return int(id_)

In [7]:
class Cell:
    def __init__(self, id_, lat, long):
        self.id = id_
        self.latitude = lat 
        self.longitude = long   
        self.coord = [[long, lat], [long+0.016, lat], [long+0.016, lat+0.009], [long,lat+0.009]]
        self.eat = list() 
        self.auto = list()
        self.purchase = list()
        self.beauty = list()
        self.service = list()
        self.entertainment = list()
        self.sport = list()
        self.religion = list()
        self.transport = list()
        self.government = list()
        self.money = list()
        self.education = list()
        self.repair = list()
        self.travel = list()

In [8]:
attrs = ['eat', 'auto', 'purchase', 'beauty', 'service', 'entertainment', 
         'sport', 'religion', 'transport', 'government', 'money', 'education', 'repair', 'travel']

In [9]:
latitudes = 55.38 + 0.009*np.array(range(1,71))
longitudes = 37.04 + 0.016*np.array(range(1,65))

In [10]:
cells = [Cell(i, lat, long) for i, (lat, long) in enumerate([(lat, long) for lat, long in product(latitudes, longitudes)])]

In [11]:
for attr in attrs:
    objs = []
    for file in tqdm(os.listdir('D:\Moscow_objects\\{}\important_objects'.format(attr))):
        with open('D:\Moscow_objects\\{}\important_objects\\'.format(attr)+file, 'rb') as f:
            objs.append(pickle.load(f))
    objs = [w for w in objs if w.lat != None ]

    for obj in objs:
        if 0 <= get_id(obj.long, obj.lat) < 4480:
            getattr(cells[get_id(obj.long, obj.lat)], attr).append(obj)

100%|███████████████████████████████████████████████████████████████████████████| 25861/25861 [03:10<00:00, 135.86it/s]


FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'D:\\Moscow_objects\\travel\\important_objects'

In [12]:
pickle.dump(cells,open('cells.data','wb'))

In [15]:
state_data = pd.DataFrame(columns=attrs)
for cell in cells:
    for attr in attrs:
        state_data.loc[cell.id, attr] = len(getattr(cell, attr))
state_data = state_data.fillna(0)

In [16]:
def transform(x):
    return  np.clip(x,np.percentile(x,98), 0)

In [17]:
state_data = state_data.apply(transform) 

In [120]:
def get_json(cells):
    
    data = { "type": "FeatureCollection",
         "features": []
       }
    
    for cell in cells:
        c =  {
                    "properties": {"name": cell.id,
                                   "popupContent": str(cell.id)},
                    "id": cell.id,
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [cell.coord]
                        }
             }
        data['features'].append(c)
        
    with open("data_file.json", "w") as write_file:
        json.dump(data, write_file)

In [121]:
get_json(cells)

In [122]:
m = folium.Map(location=(55.751244, 37.618423), zoom_start=12)

In [123]:
geo_json_data = json.load(open('data_file.json'))

In [124]:
for attr in attrs[:2]:
    colormap = linear.YlGn_09.scale(0, np.percentile(state_data[attr], 98))
    
    feature_group = FeatureGroup(name=attr)
    feature_group.layer_name = attr        
            
    # Marker(location=[cells[0].latitude, cells[0].longitude], popup=attr).add_to(feature_group)
    folium.GeoJson(
                            geo_json_data,
                            name = attr,
                            style_function=lambda feature: {
                                'fillColor': colormap(state_data.loc[feature['id'], attr]),
                                'color': 'black',
                                'weight': 1,
                                'dashArray': '5, 5',
                                'fillOpacity': 0.4,
                            }
                    
                        ).add_to(feature_group)
    
    
    """for cell in cells:
        folium.CircleMarker(location=[cell.latitude+0.0045, cell.longitude+0.008], radius = 6, 
                            popup=str(state_data.loc[cell.id, attr]),
                            #fill_color=colormap(state_data.loc[cell.id, attr]),
                            #fill=True,
                            color='black',fill_opacity=0.4).add_to(feature_group)"""
    
    
    

    feature_group.add_to(m)

In [317]:
m.save(outfile='Moscow_final_all.html')

In [316]:
m = folium.Map(location=(55.751244, 37.618423), zoom_start=12)
for attr in attrs[:-1]:
    feature_group = FeatureGroup(name=attr)
    for cell in tqdm(cells):
        #feature_group.layer_name = attr  
        
        gj = folium.GeoJson(
             data = {'type': 'FeatureCollection',
                     'features': [{'type': 'Feature',
                                   'id': cell.id,
                                   'properties': {'name': 'Alabama',
                                                  'style': {'fillColor': '#ffff00',
                                                            'color': 'black',
                                                            'weight': 2,
                                                            'dashArray': '5, 5'},
                                                  'highlight': {}
                                                 },
                                    'geometry': {'type': 'Polygon',
                                                 'coordinates': [cell.coord]}}]},
            style_function=lambda feature: {
                                    'fillColor': colormap(state_data.loc[feature['id'], attr]),
                                    'color': 'red',
                                    'weight': 0.3,
                                    'dashArray': '5, 5',
                                    'fillOpacity': 0.4,
                                }
        )
        
        html = """
                <div style="width:300px; height:300px; overflow:scroll;">
                <table>
                <caption>Перечень организаций</caption>
                  <tr>
                    <th>№</th>
                    <th>Наименование</th>
                    <th>Категория</th>
                    <th>Рейтинг</th>
                    <th>Оценки</th>
                    <th>Ссылка</th>
                  </tr>"""
        if getattr(cell, attr):
            for i, io in enumerate(getattr(cell, attr)):
                try: 
                    text = io.categories[0]
                except: 
                    text = ''
                html+=  """
                        <tr>
                        <td>{i}</td>
                        <td>{name}</td>
                        <td>{categories}</td>
                        <td>{rating}</td>
                        <td>{reviews}</td>
                        <td>{link}</td>
                        </tr>

                        """.format(i=i, link=io.link,
                                   name=io.name.replace('\'','') if io.name else '',
                                   categories=text,
                                   rating=io.rating if io.rating else '',
                                   reviews = io.reviews.split(' ')[0] if io.reviews else '')

            html += """</table>
                       </div>
                    """
            gj.add_child(folium.Popup(html=html))

            gj.add_to(feature_group)
    feature_group.add_to(m)
LayerControl().add_to(m)



  0%|                                                                                         | 0/4480 [00:00<?, ?it/s]

  0%|▏                                                                                | 8/4480 [00:00<00:58, 76.32it/s]

  0%|▎                                                                               | 17/4480 [00:00<00:54, 81.17it/s]

  1%|▍                                                                               | 25/4480 [00:00<00:56, 78.63it/s]

  1%|▌                                                                               | 33/4480 [00:00<00:59, 75.13it/s]

  1%|▋                                                                               | 39/4480 [00:00<01:03, 69.82it/s]

  1%|▊                                                                               | 46/4480 [00:00<01:04, 68.51it/s]

  1%|▉                                                                               | 54/4480 [00:00<01:03, 69.64it/s]

  1%|█▏                       

 13%|██████████                                                                     | 573/4480 [00:07<00:54, 72.09it/s]

 13%|██████████▏                                                                    | 580/4480 [00:08<00:54, 71.76it/s]

 13%|██████████▍                                                                    | 590/4480 [00:08<00:53, 72.07it/s]

 13%|██████████▌                                                                    | 598/4480 [00:08<00:54, 71.64it/s]

 14%|██████████▋                                                                    | 605/4480 [00:08<00:54, 71.51it/s]

 14%|██████████▊                                                                    | 612/4480 [00:08<00:54, 71.41it/s]

 14%|██████████▉                                                                    | 619/4480 [00:08<00:54, 71.23it/s]

 14%|███████████                                                                    | 626/4480 [00:08<00:54, 70.36it/s]

 14%|███████████▏               

 23%|█████████████████▉                                                            | 1033/4480 [00:16<00:54, 63.72it/s]

 23%|██████████████████                                                            | 1041/4480 [00:16<00:53, 63.78it/s]

 23%|██████████████████▏                                                           | 1048/4480 [00:16<00:53, 63.79it/s]

 24%|██████████████████▎                                                           | 1055/4480 [00:16<00:53, 63.71it/s]

 24%|██████████████████▍                                                           | 1062/4480 [00:16<00:53, 63.46it/s]

 24%|██████████████████▌                                                           | 1069/4480 [00:16<00:53, 63.48it/s]

 24%|██████████████████▋                                                           | 1075/4480 [00:16<00:53, 63.44it/s]

 24%|██████████████████▊                                                           | 1081/4480 [00:17<00:53, 63.42it/s]

 24%|██████████████████▉        

 33%|█████████████████████████▊                                                    | 1485/4480 [00:24<00:48, 61.58it/s]

 33%|█████████████████████████▉                                                    | 1491/4480 [00:24<00:48, 61.44it/s]

 33%|██████████████████████████                                                    | 1497/4480 [00:24<00:48, 61.39it/s]

 34%|██████████████████████████▏                                                   | 1503/4480 [00:24<00:48, 61.36it/s]

 34%|██████████████████████████▎                                                   | 1509/4480 [00:24<00:48, 61.33it/s]

 34%|██████████████████████████▍                                                   | 1515/4480 [00:24<00:48, 61.26it/s]

 34%|██████████████████████████▍                                                   | 1521/4480 [00:24<00:48, 61.24it/s]

 34%|██████████████████████████▌                                                   | 1527/4480 [00:24<00:48, 61.23it/s]

 34%|██████████████████████████▋

 42%|████████████████████████████████▉                                             | 1895/4480 [00:32<00:43, 58.80it/s]

 42%|█████████████████████████████████                                             | 1901/4480 [00:32<00:43, 58.75it/s]

 43%|█████████████████████████████████▏                                            | 1906/4480 [00:32<00:43, 58.70it/s]

 43%|█████████████████████████████████▎                                            | 1913/4480 [00:32<00:43, 58.71it/s]

 43%|█████████████████████████████████▍                                            | 1919/4480 [00:32<00:43, 58.71it/s]

 43%|█████████████████████████████████▌                                            | 1926/4480 [00:32<00:43, 58.74it/s]

 43%|█████████████████████████████████▋                                            | 1932/4480 [00:32<00:43, 58.69it/s]

 43%|█████████████████████████████████▊                                            | 1939/4480 [00:33<00:43, 58.69it/s]

 43%|███████████████████████████

...

 59%|█████████████████████████████████████████████▋                                | 2624/4480 [00:46<00:32, 56.79it/s]

 59%|█████████████████████████████████████████████▊                                | 2630/4480 [00:46<00:32, 56.78it/s]

 59%|█████████████████████████████████████████████▉                                | 2636/4480 [00:46<00:32, 56.78it/s]

 59%|██████████████████████████████████████████████                                | 2644/4480 [00:46<00:32, 56.82it/s]

 59%|██████████████████████████████████████████████▏                               | 2650/4480 [00:46<00:32, 56.80it/s]

 59%|██████████████████████████████████████████████▏                               | 2656/4480 [00:46<00:32, 56.79it/s]

 59%|██████████████████████████████████████████████▎                               | 2662/4480 [00:46<00:32, 56.75it/s]

 60%|██████████████████████████████████████████████▍                               | 2668/4480 [00:47<00:31, 56.71it/s]

 60%|███████████████████████████

 68%|████████████████████████████████████████████████████▋                         | 3028/4480 [00:54<00:25, 56.04it/s]

 68%|████████████████████████████████████████████████████▊                         | 3034/4480 [00:54<00:25, 56.01it/s]

 68%|████████████████████████████████████████████████████▉                         | 3040/4480 [00:54<00:25, 55.98it/s]

 68%|█████████████████████████████████████████████████████                         | 3045/4480 [00:54<00:25, 55.95it/s]

 68%|█████████████████████████████████████████████████████                         | 3050/4480 [00:54<00:25, 55.94it/s]

 68%|█████████████████████████████████████████████████████▏                        | 3055/4480 [00:54<00:25, 55.90it/s]

 68%|█████████████████████████████████████████████████████▎                        | 3060/4480 [00:54<00:25, 55.89it/s]

 68%|█████████████████████████████████████████████████████▎                        | 3065/4480 [00:54<00:25, 55.88it/s]

 69%|███████████████████████████

 77%|███████████████████████████████████████████████████████████▉                  | 3443/4480 [01:01<00:18, 55.56it/s]

 77%|████████████████████████████████████████████████████████████                  | 3451/4480 [01:02<00:18, 55.58it/s]

 77%|████████████████████████████████████████████████████████████▏                 | 3458/4480 [01:02<00:18, 55.59it/s]

 77%|████████████████████████████████████████████████████████████▎                 | 3464/4480 [01:02<00:18, 55.58it/s]

 78%|████████████████████████████████████████████████████████████▍                 | 3472/4480 [01:02<00:18, 55.61it/s]

 78%|████████████████████████████████████████████████████████████▌                 | 3479/4480 [01:02<00:18, 55.61it/s]

 78%|████████████████████████████████████████████████████████████▋                 | 3485/4480 [01:02<00:17, 55.59it/s]

 78%|████████████████████████████████████████████████████████████▊                 | 3491/4480 [01:02<00:17, 55.55it/s]

 78%|███████████████████████████

 87%|███████████████████████████████████████████████████████████████████▌          | 3880/4480 [01:10<00:10, 55.42it/s]

 87%|███████████████████████████████████████████████████████████████████▋          | 3886/4480 [01:10<00:10, 55.40it/s]

 87%|███████████████████████████████████████████████████████████████████▊          | 3892/4480 [01:10<00:10, 55.38it/s]

 87%|███████████████████████████████████████████████████████████████████▊          | 3898/4480 [01:10<00:10, 55.37it/s]

 87%|███████████████████████████████████████████████████████████████████▉          | 3904/4480 [01:10<00:10, 55.38it/s]

 87%|████████████████████████████████████████████████████████████████████          | 3910/4480 [01:10<00:10, 55.38it/s]

 87%|████████████████████████████████████████████████████████████████████▏         | 3917/4480 [01:10<00:10, 55.40it/s]

 88%|████████████████████████████████████████████████████████████████████▎         | 3924/4480 [01:10<00:10, 55.42it/s]

 88%|███████████████████████████

 97%|███████████████████████████████████████████████████████████████████████████▉  | 4364/4480 [01:17<00:02, 55.96it/s]

 98%|████████████████████████████████████████████████████████████████████████████  | 4370/4480 [01:18<00:01, 55.93it/s]

 98%|████████████████████████████████████████████████████████████████████████████▏ | 4376/4480 [01:18<00:01, 55.92it/s]

 98%|████████████████████████████████████████████████████████████████████████████▎ | 4384/4480 [01:18<00:01, 55.95it/s]

 98%|████████████████████████████████████████████████████████████████████████████▍ | 4391/4480 [01:18<00:01, 55.95it/s]

 98%|████████████████████████████████████████████████████████████████████████████▌ | 4397/4480 [01:18<00:01, 55.93it/s]

 98%|████████████████████████████████████████████████████████████████████████████▋ | 4403/4480 [01:18<00:01, 55.90it/s]

 98%|████████████████████████████████████████████████████████████████████████████▊ | 4410/4480 [01:18<00:01, 55.91it/s]

 99%|███████████████████████████

In [307]:
m

In [284]:
##Работает

m = folium.Map(location=(55.751244, 37.618423), zoom_start=12)
for cell in tqdm(cells[2100:2150]):
    gj = folium.GeoJson(
         data = { "type": "FeatureCollection",
                  "features": [{
                                'type': 'Polygon',
                                'coordinates': [cell.coord],
                                'id': cell.id
                              }]
                },
        style_function=lambda feature: {
                                'fillColor': colormap(state_data.loc[feature['id'], 'eat']),
                                'color': 'red',
                                'weight': 0.3,
                                'dashArray': '5, 5',
                                'fillOpacity': 0.2,
                            }
    )
    
    html = """
            <div style="width:300px; height:300px; overflow:scroll;">
            <table>
            <caption>Перечень организаций</caption>
              <tr>
                <th>№</th>
                <th>Наименование</th>
                <th>Категория</th>
                <th>Рейтинг</th>
                <th>Оценки</th>
                <th>Ссылка</th>
              </tr>"""
    if cell.eat:
        for i, io in enumerate(cell.eat):
            try: 
                text = io.categories[0]
            except: 
                text = ''
            html+=  """
                    <tr>
                    <td>{i}</td>
                    <td>{name}</td>
                    <td>{categories}</td>
                    <td>{rating}</td>
                    <td>{reviews}</td>
                    <td>{link}</td>
                    </tr>

                    """.format(i=i, link=io.link, name=io.name.replace('\'',''), categories=text,
                               rating=io.rating if io.rating else '',
                               reviews = io.reviews.split(' ')[0] if io.reviews else '')

        html += """</table>
                   </div>
                """
        gj.add_child(folium.Popup(html=html))
        
    gj.add_to(m)




  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

 18%|██████████████▉                                                                    | 9/50 [00:00<00:00, 87.76it/s]

 36%|█████████████████████████████▌                                                    | 18/50 [00:00<00:00, 85.91it/s]

 48%|███████████████████████████████████████▎                                          | 24/50 [00:00<00:00, 75.24it/s]

 58%|███████████████████████████████████████████████▌                                  | 29/50 [00:00<00:00, 66.84it/s]

 72%|███████████████████████████████████████████████████████████                       | 36/50 [00:00<00:00, 66.43it/s]

 84%|████████████████████████████████████████████████████████████████████▉             | 42/50 [00:00<00:00, 62.74it/s]

 94%|█████████████████████████████████████████████████████████████████████████████     | 47/50 [00:00<00:00, 59.58it/s]

100%|█████████████████████████

In [285]:
m

In [166]:
 self.rating = rating # "Народная" оценка уровня ослуживания 
        self.reviews = reviews
        self.lat, self.long = None, None # Широта и долгота
        self.categories = categories  # Типы объекта по внутренней классификации Яндекс.Карт
        self.link = link # Ссылка на объект 
        self.address = address  # Адрес  

IndentationError: unexpected indent (<ipython-input-166-7d0c7098d548>, line 2)

In [ ]:
feature_group = FeatureGroup(name='Some icons')
Marker(location=[45.3288, -121.6625],
       popup='Mt. Hood Meadows').add_to(feature_group)

Marker(location=[45.3311, -121.7113],
       popup='Timberline Lodge').add_to(feature_group)

feature_group.layer_name ='sdfsd'
feature_group.add_to(m)

In [ ]:
state_data.loc[feature['id'],'count']

In [ ]:
 style_function=lambda feature: {
        'fillColor': colormap(state_data.loc[feature['id'],'count']),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }

In [ ]:
m.save(outfile='Moscow_map.html')

In [ ]:
import json

In [ ]:
geo_json_data = json.load(open('data_file.json'))

In [ ]:
import branca.colormap as cm

In [ ]:
m = folium.Map(location=(55.751244, 37.618423), zoom_start=12)
с = folium.choropleth(
    geo_json_data,
    data=state_data,
    columns=['id', 'count'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
)

In [ ]:
feature_group = FeatureGroup(name='Some icons')
Marker(icon=location=[45.3288, -121.6625],
       popup='Mt. Hood Meadows').add_to(feature_group)

Marker(location=[45.3311, -121.7113],
       popup='Timberline Lodge').add_to(feature_group)

feature_group.add_to(m)
LayerControl().add_to(m)

In [ ]:
m

In [ ]:
cm.linear.OrRd_09.scale(0,101)

In [ ]:
map = folium.Map(location=(55.751244, 37.618423), zoom_start=12)

In [ ]:
for obj in objs[:4800]:
    try:
        cat = obj.categories[0]
    except:
        cat = ''
    folium.CircleMarker((obj.long, obj.lat),
                    radius=5,
                    popup='{} {} \n {} \n {}'.format(str(obj.name) or '', str(obj.rating) or '', cat or '', str(obj.link) or ''),
                    color='#3186cc',
                    fill_color='#3186cc',
                   ).add_to(map)

In [ ]:
map

In [ ]:
map = folium.Map(location=(55.751244, 37.618423), zoom_start=12)

names =  [owner.name for owner in data]
latitudes =  [owner.lat for owner in data]
longitudes = [owner.long for owner in data]
price = [owner.price for owner in data]
adress = [owner.neighbours[0].rosbank.adress for owner in data]

for lat, long, owner, adress in zip(latitudes, longitudes, data, adress):
    folium.CircleMarker((lat, long),
                    radius=5,
                    popup='{}, Росбанк: {}'.format(owner, adress),
                    color='#3186cc',
                    fill_color='#3186cc',
                   ).add_to(map)

ros_latitudes = [bank.lat for bank in rosbank_divisions]
ros_longitudes = [bank.long for bank in rosbank_divisions]
ros_adress = [bank.adress for bank in rosbank_divisions]

for lat, long, adress in zip(ros_latitudes, ros_longitudes, ros_adress):
    folium.Marker((lat, long), popup='Росбанк '+str(adress), 
                  icon=folium.Icon(color='red',icon='bank', prefix='fa') ).add_to(map)
map

In [ ]:
m.save(outfile='Moscow_map.html')